In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle, Circle
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
pd.options.display.max_rows = 100
from rich import print
import warnings
import datetime
from astropy.time import Time
import os
import astropy.units as u
from astroquery.gaia import Gaia
from astroquery.simbad import Simbad
import astropy.coordinates as coord
import astropy.units as u
import time
from gfcat_utils import parse_lightcurves_csv,parse_exposure_time,read_image
from matplotlib import gridspec
from astropy.visualization import ZScaleInterval
from matplotlib.ticker import FormatStrFormatter, MaxNLocator
from scipy.interpolate import interp1d
import tqdm

In [2]:
def angularSeparation(ra1, dec1, ra2, dec2):
    d2r = np.pi/180.
    ra2deg = 1./d2r
    d1 = dec1*d2r,
    d2 = dec2*d2r
    r1 = ra1*d2r
    r2 = ra2*d2r
    a = np.sin((d2-d1)/2.)**2.+np.cos(d1)*np.cos(d2)*np.sin((r2-r1)/2.)**2.
    r = 2*np.arcsin(np.sqrt(a))
    return r*ra2deg

def counts2flux(cps, band):
    scale = 1.4e-15 if band == 'FUV' else 2.06e-16
    return scale*cps

def counts2mag(cps, band):
    scale = 18.82 if band == 'FUV' else 20.08
    # This threw a warning if the countrate was negative which happens when
    #  the background is brighter than the source. Suppress.
    with np.errstate(invalid='ignore'):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mag = -2.5 * np.log10(cps) + scale
    return mag

def mag2counts(mag, band):
    scale = 18.82 if band == 'FUV' else 20.08
    return 10.**(-(mag-scale)/2.5)

def apcorrect1(radius, band):
    if not band in ['NUV', 'FUV']:
        print("Invalid band.")
        return
    aper = np.array([1.5, 2.3, 3.8, 6.0, 9.0, 12.8, 17.3, 30., 60., 90.])/3600.
    if radius > aper[-1]:
        return 0.
    if band == 'FUV':
        dmag = [1.65, 0.96, 0.36, 0.15, 0.1, 0.09, 0.07, 0.06, 0.03, 0.01]
    else:
        dmag = [2.09, 1.33, 0.59, 0.23, 0.13, 0.09, 0.07, 0.04, -0.00, -0.01]
        if radius > aper[-2]:
            return 0.
    if radius < aper[0]:
        return dmag[0]
    ix = np.where((aper-radius) >= 0.)
    x = [aper[ix[0][0]-1], aper[ix[0][0]]]
    y = [dmag[ix[0][0]-1], dmag[ix[0][0]]]
    m, C = np.polyfit(x, y, 1)
    return m*radius+C

In [3]:
# e37477 has two variables, so use that as a tester

In [4]:
tbl = pd.read_csv('gfcat_visit_table_crossmatched.csv',index_col=0)

for i in tqdm.tqdm(tbl.index.tolist()):
    visit = tbl.iloc[i]
    e = visit['eclipse']
    edir = str(e).zfill(5)
    epath = f"../data/lightcurves/e{edir}"
    ra,dec = visit['ra'],visit['dec']
    obs = {}
    for band in ['NUV','FUV']:
        fn = f"{epath}/e{edir}-{band[0].lower()}d-30s-photom-17_5.csv"
        try:
            expt = parse_exposure_time(f"{epath}/e{edir}-{band[0].lower()}d-30s-exptime.csv")
            lcs = parse_lightcurves_csv(fn)
            # a few eclipse have multiple variables, so we have to pick the right one
            ix = np.argmin(angularSeparation(ra,dec,
                           np.array([lc['ra'] for lc in lcs]),
                           np.array([lc['dec'] for lc in lcs])))
            lc = lcs[ix]
        except FileNotFoundError:
            #print(f'Failed to find{fn}')
            continue
        obs[band] = [lc,expt]

        # Flag certain eclipses / bands with data quality issues determined during manual QA
#        if (((band=='NUV') and (e in [3141,3225,4426,21257,21257,23279,23521,26377,30790,30872,
#                                      30875,32012,1366,7416,7927,8147,8565,28681,4406,13749,10796,
#                                      10868,28681,30696,31950,])) or
#            ((band=='FUV') and (e in [1366,3779,4406,12889,12897,3779,8256,17884,19051,23716,2648,13749,
#                                      6513,7129,7835,13112,14671,14825,21750]))):
#            tbl.loc[i,f'qa_flag_{band}']=1
        
        
        # There appears to be ramp-up phenomenon in FUV in this eclipse range
        lims = (3,None) if ((band=='FUV') and (e>=12889) and (e<=13856)) else (None,None)
        
        # add observation minimum flux
        ix = np.where(lc['cps']==np.nanmin(lc['cps'][np.where(lc['cps'][lims[0]:]!=0)]))[0][0]
        tbl.loc[i,f'cps_min_{band}']=lc['cps'][ix]
        tbl.loc[i,f'cps_min_err_{band}']=lc['cps_err'][ix]
        tbl.loc[i,f'mag_min_{band}']=counts2mag(lc['cps'][ix],band)
        tbl.loc[i,f'mag_min_err_{band}']=np.abs(
            counts2mag(lc['cps_err'][ix]+lc['cps'][ix],band)-counts2mag(lc['cps'][ix],band))
        
        # observation maximum flux
        ix = np.where(lc['cps']==np.nanmax(lc['cps'][np.where(np.isfinite(lc['cps']))]))[0][0]
        tbl.loc[i,f'cps_max_{band}']=lc['cps'][ix]
        tbl.loc[i,f'cps_max_err_{band}']=lc['cps_err'][ix]
        tbl.loc[i,f'mag_max_{band}']=counts2mag(lc['cps'][ix],band)
        tbl.loc[i,f'mag_max_err_{band}']=np.abs(
            counts2mag(lc['cps_err'][ix]+lc['cps'][ix],band)-counts2mag(lc['cps'][ix],band))

        # observation mean flux
        t = np.array(expt['expt_eff'])[lims[0]:][
            np.where(np.isfinite(lc['cps'][lims[0]:]) & (lc['cps'][lims[0]:]!=0))]
        c = lc['cps'][lims[0]:][
            np.where(np.isfinite(lc['cps'][lims[0]:]) & (lc['cps'][lims[0]:]!=0))]
        tbl.loc[i,f'cps_mean_{band}']=(c*t).sum()/t.sum()
        tbl.loc[i,f'cps_mean_err_{band}']=np.sqrt((c*t).sum())/t.sum()
        tbl.loc[i,f'mag_mean_{band}']=counts2mag((c*t).sum()/t.sum(),band)
        tbl.loc[i,f'mag_mean_err_{band}']=np.abs(
            counts2mag(np.sqrt((c*t).sum())/t.sum()+(c*t).sum()/t.sum(),band)-counts2mag((c*t).sum()/t.sum(),band))

        
        # add valid data start time, stop time, duration
        ix = np.where(np.isfinite(lc['cps']))
        tbl.loc[i,f'tstart_{band}'] = np.round(np.min(np.array(expt['t0'])[ix]))
        tbl.loc[i,f'tstop_{band}'] = np.round(np.max(np.array(expt['t0'])[ix]))
        tbl.loc[i,f'expt_raw_{band}'] = (np.round(np.max(np.array(expt['t0'])[ix])) -
                                        np.round(np.min(np.array(expt['t0'])[ix])))

    # if there is data for both bands, add the color information
    if len(obs.keys())==2:
        _,ix,_ = np.intersect1d(obs['NUV'][1]['t0'],obs['FUV'][1]['t0'][lims[0]:],return_indices=True)
        if len(ix)==0:
            print(f"Times misaligned in e{e}")
        else:
            color = (counts2flux(obs['FUV'][0]['cps'],'FUV')[ix][:-1] /
                     counts2flux(obs['NUV'][0]['cps'],'NUV')[ix][:-1])
            color_err = color * np.sqrt(
                (counts2flux(obs['NUV'][0]['cps_err'],'NUV')[ix][:-1] /
                 counts2flux(obs['NUV'][0]['cps'],    'NUV')[ix][:-1])**2 +
                (counts2flux(obs['FUV'][0]['cps_err'],'FUV')[ix][:-1] /
                 counts2flux(obs['FUV'][0]['cps'],    'FUV')[ix][:-1])**2)
            expt = obs['FUV'][1]
            ix = np.where(color==np.nanmin(color[np.where(color!=0)]))
            tbl.loc[i,'ratio_min'] = color[ix]
            tbl.loc[i,'ratio_min_err'] = color_err[ix]
            ix = np.where(color==np.nanmax(color[np.where(np.isfinite(color))]))
            tbl.loc[i,'ratio_max'] = color[ix]
            tbl.loc[i,'ratio_max_err'] = color_err[ix]
            ix = np.where(lc['cps']==np.nanmax(lc['cps'][np.where(np.isfinite(color))]))
            tbl.loc[i,'ratio_at_nuv_max'] = color[ix]
            tbl.loc[i,'ratio_at_nuv_max_err'] = color_err[ix]
#tbl[['eclipse','cps_min_FUV','cps_max_FUV','cps_min_NUV','cps_max_NUV','cps_mean_FUV','cps_mean_NUV','ratio_max']]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2065/2065 [00:22<00:00, 92.95it/s]


In [5]:
apers = [9,12.5,17.5,25,35]
for i in tqdm.tqdm(np.arange(len(tbl))):
    visit = tbl.iloc[i]
    e = visit['eclipse']
    edir = str(e).zfill(5)
    epath = f"../data/lightcurves/e{edir}"
    ra,dec = visit['ra'],visit['dec']
    for band in ['NUV','FUV']:
        try:
            expt = parse_exposure_time(f"{epath}/e{edir}-{band[0].lower()}d-30s-exptime.csv")
        except FileNotFoundError: # no data for this band
            continue
        tbl.loc[i,f'expt_eff_{band}'] = sum(expt['expt_eff'])
        for aper in apers:
            aperstr = str(aper).replace('.','_')
            fn = f"{epath}/e{edir}-{band[0].lower()}d-30s-photom-{aperstr}.csv"
            try:
                lcs = parse_lightcurves_csv(fn)
                # a few eclipse have multiple variables, so we have to pick the right one
                ix = np.argmin(angularSeparation(ra,dec,
                               np.array([lc['ra'] for lc in lcs]),
                               np.array([lc['dec'] for lc in lcs])))
                lc = lcs[ix]
            except FileNotFoundError: # no data for this aperture? but data for this band. shouldn't happen.
                print(f"Error finding {fn}")
                continue
            tbl.loc[i,f'counts_{aperstr}'] = np.sum(lc['counts'])
        aper_area = np.pi*17.5**2
        annulus_area = np.pi*35**2-np.pi*25**2
        tbl.loc[i,f'skybg_counts_{band}']=(tbl.iloc[i]['counts_35']-tbl.iloc[i]['counts_25'])*aper_area/annulus_area
        tbl.loc[i,f'skybg_counts_err_{band}']=np.sqrt(tbl.iloc[i]['counts_35']-tbl.iloc[i]['counts_25'])*aper_area/annulus_area
        tbl.loc[i,f'skybg_flux_{band}']=counts2flux(tbl.iloc[i]['counts_35']-tbl.iloc[i]['counts_25'],band)*aper_area/annulus_area
        tbl.loc[i,f'skybg_flux_err_{band}']=counts2flux(np.sqrt(tbl.iloc[i]['counts_35']-tbl.iloc[i]['counts_25']*aper_area/annulus_area),band)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████| 2065/2065 [00:51<00:00, 39.96it/s]


In [6]:
tbl

,eclipse,obj_id,ra,dec,morphology,simbad_n_match,simbad_otype,simbad_main_id,simbad_distance,simbad_distance_err_2,...,counts_35,skybg_counts_NUV,skybg_counts_err_NUV,skybg_flux_NUV,skybg_flux_err_NUV,expt_eff_FUV,skybg_counts_FUV,skybg_counts_err_FUV,skybg_flux_FUV,skybg_flux_err_FUV
0,927,3715900927.0,249.909813,41.112522,U,1,RRLyrae,V* AF Her,3149.6062992125985,141.18271167468674,...,913.206490,780.675424,19.961707,1.608191e-13,1.977242e-14,1054.934430,157.865344,8.976475,2.210115e-13,3.443420e-14
1,1413,1953301413.0,311.565113,-4.930670,F,0,Low-Mass*,2MASS J20461427-0456025,817.8621084485156,65.98240829734561,...,1016.773414,2354.825972,34.669041,4.850942e-13,2.228459e-14,939.285696,212.265016,10.408823,2.971710e-13,3.730574e-14
2,1420,1468701420.0,315.935391,-7.379244,F,1,Low-Mass*,2MASS J21034437-0722434,484.8719937936385,67.54455660357485,...,464.611123,9832.231897,70.841619,2.025440e-12,3.100942e-14,523.998065,94.460470,6.943644,1.322447e-13,2.511923e-14
3,1422,1479501422.0,343.352654,-39.793072,F,0,Galaxy,LEDA 2794348,--,--,...,3431.093699,912.604522,21.582599,1.879965e-13,1.048864e-14,NaN,NaN,NaN,NaN,NaN
4,1534,1091701534.0,259.445075,59.689607,F,1,Star,2MASS J17174656+5941240,129.1522446660123,2.0260592582583,...,407.408872,1702.620392,29.479583,3.507398e-13,1.559524e-14,537.612710,102.105720,7.219173,1.429480e-13,2.431193e-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2060,46807,--,183.176317,-26.926904,U,1,HighPM*,2MASS J12124204-2655328,43.672699322199705,0.052387960133678746,...,9822.901176,2228.415747,33.725666,4.590536e-13,1.728136e-14,NaN,NaN,NaN,NaN,NaN
2061,46818,--,184.069175,-30.364380,U,1,Star,UCAC2 19153424,1367.2409078479627,29.26927420707966,...,11364.624828,2367.059686,34.758980,4.876143e-13,1.834556e-14,NaN,NaN,NaN,NaN,NaN
2062,46828,--,183.706376,-32.431746,F,0,--,--,--,--,...,4782.771784,1008.532069,22.688578,2.077576e-13,1.192329e-14,NaN,NaN,NaN,NaN,NaN
2063,46828,--,183.710681,-32.431776,F,0,--,--,--,--,...,4512.831310,912.917138,21.586295,1.880609e-13,1.151082e-14,NaN,NaN,NaN,NaN,NaN


In [7]:
tbl.drop('morphology',axis=1,inplace=True)
#exclude_targets = pd.read_csv('exclude_targets_by_index.csv',index_col=None)
#eclipse_targets = pd.read_csv('eclipse_targets_by_index.csv',index_col=None)
#flare_targets = pd.read_csv('flare_targets_by_index.csv',index_col=None)
#for t in exclude_targets.iterrows():
#    i = t[1]['ix']
#    tbl.drop(i,inplace=True)
#for t in eclipse_targets.iterrows():
#    i = t[1]['ix']
#    tbl.loc[i,'morphology']='E'
#for t in flare_targets.iterrows():
#    e = t[1]['eclipse']
#    i = t[1]['ix']
#    print(e,i)
#    print(t['eclipse'],t.index)
#    print()
#    tbl.loc[i,'morphology']='F'

In [8]:
apers = [9,12.5,17.5,25,35]
print("Estimated aperture corrections on a")
for aper in apers:
    print(f"  =>  {aper}\" aperture is {np.round(100*apcorrect1(aper/60/60,'NUV'),1)}% in NUV and {np.round(100*apcorrect1(aper/60/60,'FUV'),1)}% in FUV")

Estimated aperture corrections on a

=>  9" aperture is 13.0% in NUV and 10.0% in FUV

=>  12.5" aperture is 9.3% in NUV and 9.1% in FUV

=>  17.5" aperture is 7.0% in NUV and 7.0% in FUV

=>  25" aperture is 5.2% in NUV and 6.4% in FUV

=>  35" aperture is 3.3% in NUV and 5.5% in FUV

In [11]:
rerun = False # if set to true, will try to rebuild the flare / eclipse / exlude list from directories

if not os.path.exists('exclude_targets_by_index.csv') or rerun:
    e,i=[],[]
    for fn in os.listdir('../data/final_qa_plots/sorted/exclude/'):
        if fn.startswith('.'):
            continue
        e+=[int(fn.split('_')[1])]
        i+=[int(fn.split('_')[2].split('.')[0])]
    exclude_list = pd.DataFrame({'eclipse':e,'ix':i})
    exclude_list.to_csv('exclude_targets_by_index.csv',index=None)
else:
    exclude_list = pd.read_csv('exclude_targets_by_index.csv',index_col=None)

if not os.path.exists('flare_targets_by_index.csv') or rerun:
    e,i=[],[]
    for fn in os.listdir('../data/final_qa_plots/sorted/flares/'):
        if fn.startswith('.'):
            continue
        e+=[int(fn.split('_')[1])]
        i+=[int(fn.split('_')[2].split('.')[0])]
    flare_list = pd.DataFrame({'eclipse':e,'ix':i})
    flare_list.to_csv('flare_targets_by_index.csv',index=None)
else:
    flare_list = pd.read_csv('flare_targets_by_index.csv',index_col=None)    

if not os.path.exists('eclipse_targets_by_index.csv') or rerun:
    e,i=[],[]
    for fn in os.listdir('../data/final_qa_plots/sorted/eclipses/'):
        if fn.startswith('.'):
            continue
        e+=[int(fn.split('_')[1])]
        i+=[int(fn.split('_')[2].split('.')[0])]
    eclipse_list = pd.DataFrame({'eclipse':e,'ix':i})
    eclipse_list.to_csv('eclipse_targets_by_index.csv',index=None)
else:
    eclipse_list = pd.read_csv('eclipse_targets_by_index.csv',index_col=None)

In [12]:
# set single-band QA flags by index, based on manual review
QA_FLAGS_NUV_IX = [37, 66, 80, 82, 96, 118, 173, 199, 238, 254, 260, 263, 432,
                   495, 603, 630, 716, 795, 812,
                   814, 881, 896, 901, 982, 1002, 1055, 1067, 1085,
                   1088, 1108, 1110, 1119, 1120, 1133, 1137, 1139,
                   1158, 1167, 1222, 1234, 1244, 1352, 1405, 1408,
                   1425, 1428, 1431, 1470, 1549, 1565, 1568, 1589,
                   1625, 1626, 1636, 1676, 1766, 1809,]
QA_FLAGS_FUV_IX = [41, 80, 173, 257, 260, 505, 506, 510, 512, 513, 741, 795,
                   838, 1554, 1571, 1632, 1636, 1645, 1704, 1716, 1748,]

In [13]:
tbl.drop(exclude_list.ix.values,inplace=True) # remove visits exluded in manual QA
tbl.loc[flare_list.ix.values,'morphology']='F'
tbl.loc[eclipse_list.ix.values,'morphology']='E'
tbl.loc[:,'qa_flag_nuv']=0
tbl.loc[:,'qa_flag_fuv']=0
tbl.loc[QA_FLAGS_NUV_IX,'qa_flag_nuv']=1
tbl.loc[QA_FLAGS_FUV_IX,'qa_flag_fuv']=1

## WRITE THE VISIT-LEVEL CATALOG!

In [14]:
tbl.replace(np.nan, '--', inplace=True)
tbl.to_csv('gfcat_visit_table_lcstats.csv')
tbl

,eclipse,obj_id,ra,dec,simbad_n_match,simbad_otype,simbad_main_id,simbad_distance,simbad_distance_err_2,simbad_distance_err_1,...,skybg_flux_NUV,skybg_flux_err_NUV,expt_eff_FUV,skybg_counts_FUV,skybg_counts_err_FUV,skybg_flux_FUV,skybg_flux_err_FUV,morphology,qa_flag_nuv,qa_flag_fuv
0,927,3715900927.0,249.909813,41.112522,1,RRLyrae,V* AF Her,3149.6062992125985,141.18271167468674,-155.0863625633242,...,0.0,0.0,1054.93443,157.865344,8.976475,0.0,0.0,--,0,0
1,1413,1953301413.0,311.565113,-4.930670,0,Low-Mass*,2MASS J20461427-0456025,817.8621084485156,65.98240829734561,-78.67724834533783,...,0.0,0.0,939.285696,212.265016,10.408823,0.0,0.0,F,0,0
2,1420,1468701420.0,315.935391,-7.379244,1,Low-Mass*,2MASS J21034437-0722434,484.8719937936385,67.54455660357485,-93.63083686476938,...,0.0,0.0,523.998065,94.46047,6.943644,0.0,0.0,F,0,0
3,1422,1479501422.0,343.352654,-39.793072,0,Galaxy,LEDA 2794348,--,--,--,...,0.0,0.0,--,--,--,--,--,F,0,0
4,1534,1091701534.0,259.445075,59.689607,1,Star,2MASS J17174656+5941240,129.1522446660123,2.0260592582583,-2.091685349043644,...,0.0,0.0,537.61271,102.10572,7.219173,0.0,0.0,F,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2059,46790,--,188.868257,-36.870892,1,"RRLyrae, Star",CRTS J123528.2-365214,7473.841554559043,1112.5183736550935,-1584.129407851242,...,0.0,0.0,--,--,--,--,--,--,0,0
2060,46807,--,183.176317,-26.926904,1,HighPM*,2MASS J12124204-2655328,43.672699322199705,0.052387960133678746,-0.05251394724312064,...,0.0,0.0,--,--,--,--,--,--,0,0
2061,46818,--,184.069175,-30.364380,1,Star,UCAC2 19153424,1367.2409078479627,29.26927420707966,-30.578495370216842,...,0.0,0.0,--,--,--,--,--,--,0,0
2063,46828,--,183.710681,-32.431776,0,--,--,--,--,--,...,0.0,0.0,--,--,--,--,--,F,0,0
